In [1]:
import pandas as pd
from epiweeks import Week
from datetime import datetime, date
from bulletin.data.notifica import Notifica
from bulletin import __file__ as __root__
from bulletin.metabase.request import download_metabase

In [2]:
notifica = Notifica()

In [3]:
notifica.download_todas_notificacoes()

In [4]:
notifica.read_todas_notificacoes()

In [5]:
todos_exames = notifica.get_casos()

In [6]:
#todos_exames.loc[~todos_exames['data_notificacao'].isnull(), 'ano'] = todos_exames['data_notificacao'].loc[~todos_exames['data_notificacao'].isnull()].apply(lambda x: Week.fromdate(x).year)
#todos_exames.loc[~todos_exames['data_notificacao'].isnull(), 'se'] = todos_exames['data_notificacao'].loc[~todos_exames['data_notificacao'].isnull()].apply(lambda x: Week.fromdate(x).week)
todos_exames = todos_exames.loc[~todos_exames['ibge_unidade_notifica'].isnull()]
todos_exames = todos_exames.loc[~todos_exames['data_notificacao'].isnull()]
todos_exames['se'] = todos_exames['data_notificacao'].apply(lambda x: Week.fromdate(x))

In [7]:
#todos_exames
tabela = todos_exames.groupby(['se','classificacao_final'])[['metodo']].count().unstack()

In [8]:
tabela = tabela.droplevel(0, axis=1)

In [9]:
for row in tabela.index.values:
    if row.year < 2020:
        tabela = tabela.drop(row)

In [10]:
tabela = tabela[[2, 3, 1, 5, 0]]

In [11]:
tabela.loc[tabela[5].isnull(), 5] = 0
tabela.loc[tabela[0].isnull(), 0] = 0
tabela[5] = tabela[5].add(tabela[0])
tabela = tabela.drop(0, axis=1)

In [12]:
tabela.to_excel('output/exames_notifica.xlsx', index = False)

In [13]:
tabela

classificacao_final,2.0,3.0,1.0,5.0
se,,,,
202001,4.0,2.0,NaN,0.0
202002,2.0,1.0,1.0,0.0
202003,3.0,NaN,NaN,0.0
202004,3.0,2.0,NaN,0.0
202005,4.0,3.0,NaN,0.0
...,...,...,...,...
202111,32389.0,23861.0,15572.0,0.0
202112,27470.0,21129.0,16178.0,0.0
202113,17266.0,15129.0,13726.0,0.0
